# Discounted Cash Flow (DCF) Analysis

This notebook estimates the intrinsic value of a company using a discounted cash
flow model based on projected free cash flows and terminal value.

In [1]:
from src.setup import *

## Step 1: Define DCF Model

We define a function that projects free cash flows over a forecast horizon and
discounts them back to present value using the company's WACC.

In [2]:

def calculate_dcf(fcf, cash, debt, discount_rate, growth, terminal_growth, years=10):
    """
    Compute the DCF Fair Value of a company.

    Parameters
    ----------
    fcf : float
        Latest Free Cash Flow.
    cash : float
        Latest cash & cash equivalents.
    debt : float
        Latest total debt.
    discount_rate : float
        Discount rate (WACC).
    growth : float
        Expected annual FCF growth (decimal, e.g., 0.05 for 5%).
    terminal_growth : float
        Long-term growth rate (decimal).
    years : int, optional
        Projection period (default 10).
    
    Returns
    -------
    fair_value : float
        Total DCF Fair Value
    discounted_cashflows : list
        Present value of FCFs per year
    discounted_terminal_value : float
        Present value of Terminal Value
    """

    
    if terminal_growth >= discount_rate:
        raise ValueError("Terminal growth must be smaller than discount rate.")
    
    discounted_cashflows = []
    fcf_year = fcf
    
    for year in range(1, years + 1):
        fcf_year *= (1 + growth) # project FCF growth
        discounted_cashflows.append(fcf_year / (1 + discount_rate)**year)
    
    terminal_value = fcf_year / (discount_rate - terminal_growth) # Terminal Value at end of projection
    discounted_terminal_value = terminal_value / (1 + discount_rate)**years
    
    # Total fair value includes cash, subtracts debt
    fair_value = sum(discounted_cashflows) + discounted_terminal_value + cash - debt
    
    return fair_value, discounted_cashflows, discounted_terminal_value




## Step 2: Define Input Assumptions

We extract the latest financial data and define assumptions for growth,
terminal growth, and discount rate (WACC).


In [3]:
# Inputs from your prepared data
try:
    latest_fcf = annual_avg[f"{TICKER}: Annual Free Cash Flow"].iloc[-1]
    latest_cash = annual_avg[f"{TICKER}: Quarterly Cash & Cash Equivalents"].iloc[-1]
    latest_debt = balance_sheet.loc["Total Debt"].iloc[0]
except:
    latest_fcf = annual_avg[f"{TICKER}: Annual Free Cash Flow"].iloc[-2]
    latest_cash = annual_avg[f"{TICKER}: Quarterly Cash & Cash Equivalents"].iloc[-1]
    latest_debt = balance_sheet.loc["Total Debt"].iloc[0]

# User-defined assumptions
expected_growth = 0.05           # 5% annual FCF growth
terminal_growth = 0.03           # 3% long-term growth

# Compute WACC for discounting
equity_ratio_wacc = market_cap / (market_cap + latest_debt)
debt_ratio_wacc = latest_debt / (market_cap + latest_debt)
equity_cost = RISK_FREE_RATE + BETA * market_risk_premium
debt_cost = (RISK_FREE_RATE + CREDIT_SPREAD) * (1 - TAX_RATE/100)
wacc = equity_ratio_wacc * equity_cost + debt_ratio_wacc * debt_cost
discount_rate = wacc


## Step 3: Run DCF Model

We compute the fair value, discounted cash flows, and value per share.

In [4]:
# Calculate DCF
fair_value, discounted_cf, discounted_terminal = calculate_dcf(
    fcf=latest_fcf,
    cash=latest_cash,
    debt=latest_debt,
    discount_rate=discount_rate,
    growth=expected_growth,
    terminal_growth=terminal_growth,
    years=10
)

# Prepare output as DataFrame
df_dcf = pd.DataFrame({
    "Year": list(range(1, 11)) + ["Terminal"],
    "Discounted FCF ($)": discounted_cf + [discounted_terminal]
})
df_dcf.set_index("Year", inplace=True)
df_dcf["Discounted FCF ($)"] = df_dcf["Discounted FCF ($)"].apply(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)

# Value per share
value_per_share = fair_value / shares_outstanding


## Step 4: Results

Below are the discounted cash flows, fair value, and intrinsic value per share.


In [5]:
# =========================
# Output
# =========================
print("- Discounted Cash Flow (DCF) Analysis -\n")
print(f"Discount Rate (WACC): {discount_rate:.2%}")
print(f"Value per Share: ${value_per_share:,.2f}")
print(f"Calculated Fair Value: ${fair_value:,.0f}")
print(f"Latest Market Cap: ${market_cap:,.0f}\n")
print()
print("DCF Discounted Cashflows over 10 years:")
display(df_dcf)

- Discounted Cash Flow (DCF) Analysis -

Discount Rate (WACC): 9.11%
Value per Share: $251.69
Calculated Fair Value: $1,870,685,852,954
Latest Market Cap: $3,301,143,478,272


DCF Discounted Cashflows over 10 years:


,Discounted FCF ($)
Year,
1,"91,778,287,560.14"
2,"88,318,755,076.25"
3,"84,989,627,782.14"
4,"81,785,990,125.31"
5,"78,703,111,842.36"
6,"75,736,440,974.55"
7,"72,881,597,146.76"
8,"70,134,365,099.73"
9,"67,490,688,466.07"


## Step 5: Historical DCF (Optional)

This section computes historical fair values using today’s assumptions
for visualization and comparison purposes.

In [6]:
def historical_dcf(annual_avg, growth, terminal_growth, discount_rate):
    """
    Compute DCF Fair Value historically for each year.
    Returns a DataFrame with Fair Values per historical year.

    Parameters
    ----------
    annual_avg : pandas.DataFrame
        Annualized financial data including Free Cash Flow, Cash, and Debt.
    growth : float
        Expected annual Free Cash Flow growth (decimal, e.g., 0.05 for 5%).
    terminal_growth : float
        Long-term growth rate (decimal, e.g., 0.03 for 3%).
    discount_rate : float
        Discount rate to apply (typically WACC).

    Returns
    -------
    pandas.DataFrame
        DataFrame with historical years as index and the calculated
        DCF Fair Value for each year in a column named
        "Historical Fair Value ($)".

    Note:
        Historical fair values are **discounted using today's discount rate and growth assumptions**.
        They **do not represent the actual fair value at that historical date** and should only
        be used for visualization or illustrative purposes.

    """

    historical_cash = annual_avg[f"{TICKER}: Quarterly Cash & Cash Equivalents"].tolist()
    historical_debt = annual_avg[f"{TICKER}: Quarterly Long Term Debt"].tolist()
    historical_fcf = annual_avg[f"{TICKER}: Annual Free Cash Flow"].tolist()
    years = annual_avg.index.tolist()
    
    fair_values = []
    for i in range(len(years)):
        fv, _, _ = calculate_dcf(
            fcf=historical_fcf[i],
            cash=historical_cash[i],
            debt=historical_debt[i],
            growth=growth,
            terminal_growth=terminal_growth,
            discount_rate=discount_rate,
            years=10
        )
        fair_values.append(fv)
    
    df_hist = pd.DataFrame({
        "Year": years,
        "Historical Fair Value ($)": fair_values
    })
    df_hist.set_index("Year", inplace=True)
    df_hist["Historical Fair Value ($)"] = df_hist["Historical Fair Value ($)"].apply(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)
    return df_hist

## Step 6: Historical DCF Results

In [7]:
# Example historical usage
df_historical = historical_dcf(annual_avg, expected_growth, terminal_growth, discount_rate)
print("\nHistorical DCF Fair Values:")
display(df_historical)


Historical DCF Fair Values:


,Historical Fair Value ($)
Year,
2005,"182,264,534,280.79"
2006,"262,231,613,176.18"
2007,"402,549,495,123.63"
2008,"345,332,219,059.51"
2009,"390,106,914,289.36"
2010,"481,462,977,401.61"
2011,"557,125,585,034.83"
2012,"586,151,498,524.89"
2013,"500,491,326,474.40"
